In [6]:
import gym
from gym.envs.registration import register
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from keras import layers, models

# deterministic world
register(
    id='FrozenLake-v3',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False}
)
env = gym.make('FrozenLake-v3')


def one_hot(total, state):  #상태를 one hot 인코딩합니다.예) 0번째 상태 1000000000000000
    return np.identity(total)[state:state+1]           #예) 1번째 상태 0100000000000000


def ANN_seq_func(Nin, Nout):
    model = models.Sequential()
    model.add(layers.Dense(Nout,    input_shape=(Nin,)))
    model.compile(loss='mse',
                  optimizer='Adam'  )

    return model

C:\Users\admin\anaconda3\Lib\site-packages\gym\envs\registration.py:498: UserWarning: WARN: Overriding environment FrozenLake-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [ ]:
####################################################
def main() :
    dis = .99
    num_episodes = 20
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    rList = []

    model =ANN_seq_func(num_states,num_actions)

    for i in range(num_episodes):
        state = env.reset()[0]
        e = 1 / ((i / 50) + 10)
        rAll = 0
        done = False

        while not done :
            Qs = model.predict (one_hot(num_states, state   ))
 
            if np.random.rand(1) < e:
                a = env.action_space.sample()
            else:
                a = np.argmax(Qs)

            new_state, reward, done, _ ,_ = env.step(a)
            if done:
                Qs[0, a] = reward
            else:
                Qs1 = model.predict (one_hot(num_states,  new_state  ))
                Qs[0, a] = reward + dis * np.max( Qs1  )


            model.fit(one_hot(num_states,state  ),  Qs  ,  verbose=0 )

            rAll += reward
            state = new_state

        rList.append(rAll)

    print("Success rate: " + str(sum(rList) / num_episodes))
    plt.bar(range(len(rList)), rList, color='b', alpha=0.4)
    plt.show()


if __name__ == '__main__':
    main()

1/1 [==============================] - 0s 21ms/step

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 16ms/step
